In [95]:
import psycopg2
import requests
import re

In [9]:
with open('/Users/crupley/.api/walkscore.txt') as f:
    wskey = f.readline().strip()

In [ ]:
lat = 37.7700103
lon = -122.4535951

In [60]:
requrl = 'http://api.walkscore.com/score'
payload = {'wsapikey':wskey,
           'lat': lat,
           'lon': lon,
           'format': 'json'}
response = requests.get(requrl, params = payload)

In [15]:
response.json()

{u'description': u"Walker's Paradise",
 u'help_link': u'https://www.redfin.com/how-walk-score-works',
 u'logo_url': u'https://cdn.walk.sc/images/api-logo.png',
 u'more_info_icon': u'https://cdn.walk.sc/images/api-more-info.gif',
 u'more_info_link': u'https://www.redfin.com/how-walk-score-works',
 u'snapped_lat': 37.77,
 u'snapped_lon': -122.454,
 u'status': 1,
 u'updated': u'2015-06-01 22:34:36.166320',
 u'walkscore': 93,
 u'ws_link': u'https://www.walkscore.com/score/loc/lat=37.7700103/lng=-122.4535951/?utm_source=crupley.com&utm_medium=ws_api&utm_campaign=ws_api'}

In [61]:
data = pd.Series(response.json())
data['searched_lat'] = lat
data['searched_lon'] = lon
data

help_link               https://www.redfin.com/how-walk-score-works
logo_url                    https://cdn.walk.sc/images/api-logo.png
more_info_icon         https://cdn.walk.sc/images/api-more-info.gif
more_info_link          https://www.redfin.com/how-walk-score-works
snapped_lat                                                  -0.051
snapped_lon                                                 -0.0645
status                                                            2
ws_link           https://www.walkscore.com/score/loc/lat=-0.050...
searched_lat                                              -0.050653
searched_lon                                             -0.0640791
dtype: object

In [27]:
for i in list(data.keys()): print i

description
help_link
logo_url
more_info_icon
more_info_link
snapped_lat
snapped_lon
status
updated
walkscore
ws_link
searched_lat
searched_lon


In [35]:
data.values

array([u"Walker's Paradise",
       u'https://www.redfin.com/how-walk-score-works',
       u'https://cdn.walk.sc/images/api-logo.png',
       u'https://cdn.walk.sc/images/api-more-info.gif',
       u'https://www.redfin.com/how-walk-score-works', 37.77, -122.454, 1,
       u'2015-06-01 22:34:36.166320', 93,
       u'https://www.walkscore.com/score/loc/lat=37.7700103/lng=-122.4535951/?utm_source=crupley.com&utm_medium=ws_api&utm_campaign=ws_api',
       37.7700103, -122.4535951], dtype=object)

In [36]:
ncols = data.shape[0]
q_string = 	'''
    INSERT INTO walkscore_raw (description,
                               help_link,
                               logo_url,
                               more_info_icon,
                               more_info_link,
                               snapped_lat,
                               snapped_lon,
                               status,
                               updated,
                               walkscore,
                               ws_link,
                               searched_lat,
                               searched_lon)
        VALUES (%s)''' % ('%s, ' * (ncols - 1) + '%s')

In [38]:
conn = psycopg2.connect(dbname='hoodie', user='postgres', host='/tmp')
c = conn.cursor()

c.execute(q_string, data.values)

conn.commit()
conn.close()

In [63]:
%run ../code/geo.py

In [64]:
print g.shape
g[:5,:]

(5476, 2)


array([[  37.719347  , -122.51807913],
       [  37.72071748, -122.51807913],
       [  37.72208796, -122.51807913],
       [  37.72345844, -122.51807913],
       [  37.72482893, -122.51807913]])

In [54]:
%run ../code/scrape.py

In [66]:
for lat, lon in g[5:,:]:
     scrape_walkscore(lat, lon)

# greatschools

In [2]:
with open('/Users/crupley/.api/greatschools.txt') as f:
    gskey = f.readline().strip()

In [18]:
# browse schools
state = 'CA'
city = 'San-Francisco'

requrl = 'http://api.greatschools.org/schools/' + state + '/' + city
payload = {'key': gskey,
           'limit': 1000}

In [24]:
# nearby schools
state = 'CA'
city = 'San+Francisco'

requrl = 'http://api.greatschools.org/schools/nearby'
payload = {'key': gskey,
           'state': state,
           'city': city,
           'limit': 1000}

In [27]:
response = requests.get(requrl, params = payload)
response

<Response [200]>

In [28]:
response.url

u'http://api.greatschools.org/schools/nearby?city=San%2BFrancisco&state=CA&limit=1000&key=9dccbhobxcvc6xkulcedegxx'

In [29]:
response.text

u'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><schools><school><gsId>24793</gsId><name>Five Keys Adult School (Sf Sheriff\'s)</name><type>charter</type><gradeRange>9-12</gradeRange><enrollment>95</enrollment><parentRating>5</parentRating><city>San Francisco</city><state>CA</state><districtId>717</districtId><district>San Francisco Unified School District</district><districtNCESId>0634410</districtNCESId><address>70 Oak Grove, \nSan Francisco, CA  94107</address><phone>(415) 734-3310</phone><fax>(415) 734-3314</fax><website>http://www.fivekeyscharter.org</website><ncesId>063441012373</ncesId><lat>37.775295</lat><lon>-122.42051</lon><overviewLink>http://www.greatschools.org/california/san-francisco/24793-Five-Keys-Adult-School-(Sf-Sheriff\'s)/?s_cid=gsapi</overviewLink><ratingsLink>http://www.greatschools.org/school/rating.page?state=CA&amp;id=24793&amp;s_cid=gsapi</ratingsLink><reviewsLink>http://www.greatschools.org/school/parentReviews.page?state=CA&amp;id=24793&amp;s_cid=g

In [6]:
from xml.etree import ElementTree

In [30]:
tree = ElementTree.fromstring(response.content)

In [31]:
len(tree)

275

In [32]:
print tree.tag
print tree.attrib

schools
{}


In [33]:
tree[0][0].tag

'gsId'

In [34]:
for att in tree[0]:
    print att.tag, att.text

gsId 24793
name Five Keys Adult School (Sf Sheriff's)
type charter
gradeRange 9-12
enrollment 95
parentRating 5
city San Francisco
state CA
districtId 717
district San Francisco Unified School District
districtNCESId 0634410
address 70 Oak Grove, 
San Francisco, CA  94107
phone (415) 734-3310
fax (415) 734-3314
website http://www.fivekeyscharter.org
ncesId 063441012373
lat 37.775295
lon -122.42051
overviewLink http://www.greatschools.org/california/san-francisco/24793-Five-Keys-Adult-School-(Sf-Sheriff's)/?s_cid=gsapi
ratingsLink http://www.greatschools.org/school/rating.page?state=CA&id=24793&s_cid=gsapi
reviewsLink http://www.greatschools.org/school/parentReviews.page?state=CA&id=24793&s_cid=gsapi
distance 0.07
schoolStatsLink http://www.greatschools.org/modperl/achievement/CA/24793


In [35]:
d = dict()
for school in tree:
    for att in school:
        value = att.text
        if type(value) == None: value = -1
        d[att.tag] = d.get(att.tag, list()) + [value]
d

{'address': ['70 Oak Grove, \nSan Francisco, CA  94107',
  '70 Oak Grove, \nSan Francisco, CA  94107',
  '150 Oak Street, \nSan Francisco, CA  94102',
  '150 Oak Street, \nSan Francisco, CA  94102',
  '95 Gough Street, \nSan Francisco, CA  94102',
  '250 10th Street, \nSan Francisco, CA  94103',
  '250 10th Street, \nSan Francisco, CA  94103',
  '214 Haight Street, \nSan Francisco, CA  94102',
  '727 Golden Gate Avenue, \nSan Francisco, CA  94102',
  '250 Valencia Street, \nSan Francisco, CA  94103',
  '627 Turk Street, \nSan Francisco, CA  94102',
  '1245 Folsom Street, \nSan Francisco, CA  94105',
  '1245 folsom st, \nSan Francisco, CA  94103',
  '380 Webster Street, \nSan Francisco, CA  94117',
  '1575 15th Street, \nSan Francisco, CA  94103',
  '175 Golden Gate Avenue, \nSan Francisco, CA  94102',
  'Preschool campus: 728 20th Street, K-8 campus: 735 Fell Street, \nSan Francisco, CA  94107',
  '735 Fell St, \nSan Francisco, CA  94117',
  '1055 Ellis Street, \nSan Francisco, CA  941

In [144]:
for key in d.keys():
    print key, len(d[key])

schoolStatsLink 200
city 200
ratingsLink 200
district 109
parentRating 167
lon 200
gsId 200
state 200
type 200
website 200
overviewLink 200
fax 200
gsRating 94
address 200
lat 200
name 200
phone 200
reviewsLink 200
enrollment 177
districtId 109
districtNCESId 109
ncesId 200
gradeRange 200


In [156]:
print school.items()

[]


# US Census
## Age/gender

In [40]:
df = pd.read_csv('../data/uscensus/p12/DEC_10_SF1_P12.csv', skiprows=1)
print df.shape
df.head(1)

(7386, 52)


,Id,Id2,Geography,Total:,Male:,Male: - Under 5 years,Male: - 5 to 9 years,Male: - 10 to 14 years,Male: - 15 to 17 years,Male: - 18 and 19 years,...,Female: - 50 to 54 years,Female: - 55 to 59 years,Female: - 60 and 61 years,Female: - 62 to 64 years,Female: - 65 and 66 years,Female: - 67 to 69 years,Female: - 70 to 74 years,Female: - 75 to 79 years,Female: - 80 to 84 years,Female: - 85 years and over
0,1000000US060750101001000,60750101001000,"Block 1000, Block Group 1, Census Tract 101, S...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
df.Geography[0]

'Block 1000, Block Group 1, Census Tract 101, San Francisco County, California'

In [44]:
def splitgeo(geo):
    s = geo.split(',')
    block = s[0].split(' ')[-1]
    blockgroup = s[1].split(' ')[-1]
    tract = s[2].split(' ')[-1]
    return block, blockgroup, tract
    
splitgeo(df.Geography[0])

('1000', '1', '101')

In [49]:
df.Geography[:2].apply(splitgeo)

0    (1000, 1, 101)
1    (1001, 1, 101)
Name: Geography, dtype: object

In [50]:
df['geovalues'] = df.Geography.apply(splitgeo)

In [51]:
cols = ['Block', 'Block_Group', 'Tract']
for n, col in enumerate(cols):
    df[col] = df.geovalues.apply(lambda x: x[n])
df.head(1)

,Id,Id2,Geography,Total:,Male:,Male: - Under 5 years,Male: - 5 to 9 years,Male: - 10 to 14 years,Male: - 15 to 17 years,Male: - 18 and 19 years,...,Female: - 65 and 66 years,Female: - 67 to 69 years,Female: - 70 to 74 years,Female: - 75 to 79 years,Female: - 80 to 84 years,Female: - 85 years and over,geovalues,Block,Block_Group,Tract
0,1000000US060750101001000,60750101001000,"Block 1000, Block Group 1, Census Tract 101, S...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,"(1000, 1, 101)",1000,1,101


In [67]:
%run ../code/makedbs.py
df = pd.read_csv('../data/uscensus/p12/DEC_10_SF1_P12.csv', skiprows=1)
df = clean_usc_age_gender(df)
df.head(1)

,Block,Block_Group,Tract,Total,M,M_U5,M_5_9,M_10_14,M_15_17,M_18_19,...,F_50_54,F_55_59,F_60_61,F_62_64,F_65_66,F_67_69,F_70_74,F_75_79,F_80_84,F_85_over
0,101,1,1000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [68]:
for i in df.columns: print i

Block
Block_Group
Tract
Total
M
M_U5
M_5_9
M_10_14
M_15_17
M_18_19
M_20
M_21
M_22_24
M_25_29
M_30_34
M_35_39
M_40_44
M_45_49
M_50_54
M_55_59
M_60_61
M_62_64
M_65_66
M_67_69
M_70_74
M_75_79
M_80_84
M_85_over
F
F_U5
F_5_9
F_10_14
F_15_17
F_18_19
F_20
F_21
F_22_24
F_25_29
F_30_34
F_35_39
F_40_44
F_45_49
F_50_54
F_55_59
F_60_61
F_62_64
F_65_66
F_67_69
F_70_74
F_75_79
F_80_84
F_85_over


In [76]:
df.Tract.astype('int').unique()

array([1000, 1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010,
       1011, 1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021,
       1022, 1023, 1024, 1025, 1026, 1027, 2000, 2001, 2002, 2003, 2004,
       2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015,
       2016, 2017, 2018, 2019, 3000, 3001, 3002, 3003, 3004, 3005, 3006,
       3007, 3008, 3009, 3010, 3011, 3012, 3013, 3014, 4000, 4001, 4002,
       4003, 4004, 2020, 2021, 1028, 1029, 1030, 1031, 2022, 2023, 2024,
       2025, 2026, 2027, 2028, 2029, 4005, 4006, 4007, 3015, 3016, 3017,
       3018, 3019, 4008, 4009, 4010, 5000, 5001, 5002, 5003, 5004, 5005,
       5006, 5007, 4011, 5008, 5009, 5010, 1032, 1033, 1034, 1035, 1036,
       1037, 1038, 1039, 1040, 1041, 1042, 1043, 1044, 1045, 1046, 1047,
       1048, 1049, 1050, 1051, 1052, 1053, 1054, 1055, 1056, 1057, 1058,
       1059, 1060, 1061, 2030, 2031, 2032, 2033, 2034, 2035, 2036, 2037,
       2038, 2039, 2040, 2041, 2042, 2043, 2044, 20

## US Census household size

In [88]:
%run ../code/makedbs.py
df = load_usc_household()
print df.shape
df.head(1)

(7386, 11)


,Id,Id2,Geography,Total:,1-person household,2-person household,3-person household,4-person household,5-person household,6-person household,7-or-more-person household
0,1000000US060750101001000,60750101001000,"Block 1000, Block Group 1, Census Tract 101, S...",0,0,0,0,0,0,0,0


In [99]:
%run ../code/makedbs.py
df = load_usc_household()
df = clean_usc_household(df)
df.head(1)

,Block,Block_Group,Tract,Total,p1,p2,p3,p4,p5,p6,p7
0,101,1,1000,0,0,0,0,0,0,0,0


In [96]:
fnames = []
for name in cnames:
    pp = ''
    if re.match('[0-9]', name): pp = 'p'
    fnames.append(pp + name)
    

In [97]:
fnames

['Block',
 'Block_Group',
 'Tract',
 'Total',
 'p1_person',
 'p2_person',
 'p3_person',
 'p4_person',
 'p5_person',
 'p6_person',
 'p7_person']

In [100]:
for i in df.columns: print i

Block
Block_Group
Tract
Total
p1
p2
p3
p4
p5
p6
p7


In [ ]:
%run ../code/makedbs.py